In [3]:
import pandas as pd
import numpy as np
from pyvis.network import Network

In [27]:
data = pd.read_csv("C:/Users/conno_52qz545/CS3220 AI/cs3220_2025f/data/game-of-thrones-battles.csv")
data.head()


,name,year,battle_number,attacker_king,defender_king,attacker_1,attacker_2,attacker_3,attacker_4,defender_1,...,major_death,major_capture,attacker_size,defender_size,attacker_commander,defender_commander,summer,location,region,note
0,Battle of the Golden Tooth,298,1,Joffrey/Tommen Baratheon,Robb Stark,Lannister,NaN,NaN,NaN,Tully,...,1.0,0.0,15000.0,4000.0,Jaime Lannister,"Clement Piper, Vance",1.0,Golden Tooth,The Westerlands,NaN
1,Battle at the Mummer's Ford,298,2,Joffrey/Tommen Baratheon,Robb Stark,Lannister,NaN,NaN,NaN,Baratheon,...,1.0,0.0,NaN,120.0,Gregor Clegane,Beric Dondarrion,1.0,Mummer's Ford,The Riverlands,NaN
2,Battle of Riverrun,298,3,Joffrey/Tommen Baratheon,Robb Stark,Lannister,NaN,NaN,NaN,Tully,...,0.0,1.0,15000.0,10000.0,"Jaime Lannister, Andros Brax","Edmure Tully, Tytos Blackwood",1.0,Riverrun,The Riverlands,NaN
3,Battle of the Green Fork,298,4,Robb Stark,Joffrey/Tommen Baratheon,Stark,NaN,NaN,NaN,Lannister,...,1.0,1.0,18000.0,20000.0,"Roose Bolton, Wylis Manderly, Medger Cerwyn, H...","Tywin Lannister, Gregor Clegane, Kevan Lannist...",1.0,Green Fork,The Riverlands,NaN
4,Battle of the Whispering Wood,298,5,Robb Stark,Joffrey/Tommen Baratheon,Stark,Tully,NaN,NaN,Lannister,...,1.0,1.0,1875.0,6000.0,"Robb Stark, Brynden Tully",Jaime Lannister,1.0,Whispering Wood,The Riverlands,NaN


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38 entries, 0 to 37
Data columns (total 25 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   name                38 non-null     object 
 1   year                38 non-null     int64  
 2   battle_number       38 non-null     int64  
 3   attacker_king       36 non-null     object 
 4   defender_king       35 non-null     object 
 5   attacker_1          38 non-null     object 
 6   attacker_2          10 non-null     object 
 7   attacker_3          3 non-null      object 
 8   attacker_4          2 non-null      object 
 9   defender_1          37 non-null     object 
 10  defender_2          2 non-null      object 
 11  defender_3          0 non-null      float64
 12  defender_4          0 non-null      float64
 13  attacker_outcome    37 non-null     object 
 14  battle_type         37 non-null     object 
 15  major_death         37 non-null     float64
 16  major_capt

In [6]:
battles_df=data.loc[:,['name','attacker_king','defender_king','attacker_size','defender_size']]
battles_df.head()

,name,attacker_king,defender_king,attacker_size,defender_size
0,Battle of the Golden Tooth,Joffrey/Tommen Baratheon,Robb Stark,15000.0,4000.0
1,Battle at the Mummer's Ford,Joffrey/Tommen Baratheon,Robb Stark,NaN,120.0
2,Battle of Riverrun,Joffrey/Tommen Baratheon,Robb Stark,15000.0,10000.0
3,Battle of the Green Fork,Robb Stark,Joffrey/Tommen Baratheon,18000.0,20000.0
4,Battle of the Whispering Wood,Robb Stark,Joffrey/Tommen Baratheon,1875.0,6000.0


In [7]:
battles_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38 entries, 0 to 37
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   name           38 non-null     object 
 1   attacker_king  36 non-null     object 
 2   defender_king  35 non-null     object 
 3   attacker_size  24 non-null     float64
 4   defender_size  19 non-null     float64
dtypes: float64(2), object(3)
memory usage: 1.6+ KB


In [8]:
battles_df_cleaned=battles_df.dropna()
battles_df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16 entries, 0 to 37
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   name           16 non-null     object 
 1   attacker_king  16 non-null     object 
 2   defender_king  16 non-null     object 
 3   attacker_size  16 non-null     float64
 4   defender_size  16 non-null     float64
dtypes: float64(2), object(3)
memory usage: 768.0+ bytes


In [9]:
print(f"Attacking kings: {battles_df_cleaned.attacker_king.unique()}")

Attacking kings: ['Joffrey/Tommen Baratheon' 'Robb Stark' 'Stannis Baratheon']


In [10]:
print(f"Defending kings: {battles_df_cleaned.defender_king.unique()}")

Defending kings: ['Robb Stark' 'Joffrey/Tommen Baratheon' 'Balon/Euron Greyjoy'
 'Renly Baratheon' 'Mance Rayder']


In [11]:
net5kings = Network(heading="Task1. Building Interactive Network of battles of War of 5",
                    bgcolor = "#242020",
                    font_color = "white",
                    height = "1000px",
                    width = "100%",
                    directed = True, # we have directed graph
                    notebook = True,
                    cdn_resources = "remote"
)

In [12]:
type(data['attacker_king'].values)

numpy.ndarray

In [13]:
Kings_list=np.unique(np.concatenate((battles_df_cleaned['attacker_king'].values,battles_df_cleaned['defender_king'].values),0))
Kings_list

array(['Balon/Euron Greyjoy', 'Joffrey/Tommen Baratheon', 'Mance Rayder',
       'Renly Baratheon', 'Robb Stark', 'Stannis Baratheon'], dtype=object)

In [14]:
net5kings.add_nodes(Kings_list, title=[str(Kings_list) for node in Kings_list])
net5kings.nodes

[{'color': '#97c2fc',
  'title': "['Balon/Euron Greyjoy' 'Joffrey/Tommen Baratheon' 'Mance Rayder'\n 'Renly Baratheon' 'Robb Stark' 'Stannis Baratheon']",
  'id': 'Balon/Euron Greyjoy',
  'label': 'Balon/Euron Greyjoy',
  'shape': 'dot',
  'font': {'color': 'white'}},
 {'color': '#97c2fc',
  'title': "['Balon/Euron Greyjoy' 'Joffrey/Tommen Baratheon' 'Mance Rayder'\n 'Renly Baratheon' 'Robb Stark' 'Stannis Baratheon']",
  'id': 'Joffrey/Tommen Baratheon',
  'label': 'Joffrey/Tommen Baratheon',
  'shape': 'dot',
  'font': {'color': 'white'}},
 {'color': '#97c2fc',
  'title': "['Balon/Euron Greyjoy' 'Joffrey/Tommen Baratheon' 'Mance Rayder'\n 'Renly Baratheon' 'Robb Stark' 'Stannis Baratheon']",
  'id': 'Mance Rayder',
  'label': 'Mance Rayder',
  'shape': 'dot',
  'font': {'color': 'white'}},
 {'color': '#97c2fc',
  'title': "['Balon/Euron Greyjoy' 'Joffrey/Tommen Baratheon' 'Mance Rayder'\n 'Renly Baratheon' 'Robb Stark' 'Stannis Baratheon']",
  'id': 'Renly Baratheon',
  'label': 'Ren

In [15]:
edges = battles_df_cleaned.loc[:,["attacker_king", "defender_king"]].values.tolist()
edges[:]

[['Joffrey/Tommen Baratheon', 'Robb Stark'],
 ['Joffrey/Tommen Baratheon', 'Robb Stark'],
 ['Robb Stark', 'Joffrey/Tommen Baratheon'],
 ['Robb Stark', 'Joffrey/Tommen Baratheon'],
 ['Robb Stark', 'Joffrey/Tommen Baratheon'],
 ['Robb Stark', 'Balon/Euron Greyjoy'],
 ['Joffrey/Tommen Baratheon', 'Robb Stark'],
 ['Robb Stark', 'Joffrey/Tommen Baratheon'],
 ['Stannis Baratheon', 'Renly Baratheon'],
 ['Joffrey/Tommen Baratheon', 'Robb Stark'],
 ['Robb Stark', 'Joffrey/Tommen Baratheon'],
 ['Stannis Baratheon', 'Joffrey/Tommen Baratheon'],
 ['Joffrey/Tommen Baratheon', 'Robb Stark'],
 ['Stannis Baratheon', 'Mance Rayder'],
 ['Stannis Baratheon', 'Balon/Euron Greyjoy'],
 ['Stannis Baratheon', 'Joffrey/Tommen Baratheon']]

In [20]:
edges_unique = []
for item in edges:
    if item not in edges_unique:
        edges_unique.append(item)
edges_unique

[['Joffrey/Tommen Baratheon', 'Robb Stark'],
 ['Robb Stark', 'Joffrey/Tommen Baratheon'],
 ['Robb Stark', 'Balon/Euron Greyjoy'],
 ['Stannis Baratheon', 'Renly Baratheon'],
 ['Stannis Baratheon', 'Joffrey/Tommen Baratheon'],
 ['Stannis Baratheon', 'Mance Rayder'],
 ['Stannis Baratheon', 'Balon/Euron Greyjoy']]

In [26]:
df = pd.DataFrame(edges)

test = df.groupby(['attacker_king','defender_king'])['attacker_king'].count()

KeyError: 'attacker_king'

In [23]:
df = pd.DataFrame(edges)

edge_weight = df.groupby(["attacker_king", "defender_king"]).agg(count=("attacker_king","defender_king")).reset_index
print(edge_weight)

KeyError: 'attacker_king'